<i><b>Public-AI</b></i>
#### week 2. 협업 필터링

# 부록. 대규모 서비스를 위한 Collaborative Filtering 


이번 섹션은 [Deview 2015 하용호님 발표](https://www.slideshare.net/deview/261-52784785)의 발표 내용을 기반으로 동일한 내용을 Python 코드로서 풀어낸 내용입니다. **코끼리 냉장고에 집어넣기 : 실시간 추천엔진을 노트북에서 돌게 만들어보자** 라는 주제로 발표된 추천 엔진 관련 발표는 분산 환경인 하둡이나 스파크를 쓰지 않고도, 노트북 수준으로도 충분히  대용량의 추천 엔진을 디자인할 수 있음을 보여주는 예제 프로젝트입니다. 

### _Objective_ 
* 실제 추천 서버로 동작하기 위해서는 대규모의 요청에서도 빠르게 응답할 수 있어야 합니다.
* 이를 위해서 `MinHash` 알고리즘과 `Redis` 캐시 서버를 이용한 방법론을 살펴보도록 하겠습니다.

In [ ]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.utils import get_file
tqdm.pandas()

### 사전 준비 

튜토리얼을 따라하기 위해서는 아래의 요소들이 설치되어 있어야 합니다.

1. minHash 알고리즘을 계산하는 파이썬 라이브러리(datasketch) 설치
    ````shell
    pip install datasketch
    ````

2. Cache Database인 `Redis` 설치하기
    
    * [Redis 설치 가이드](https://redis.io/topics/quickstart)

    ````shell
    pip install redis # Python Binding Library
    ````


# [ 데이터 불러오기 ]

----

### (1) 데이터 가져오기 

In [ ]:
ROOT_URL = "https://pai-datasets.s3.ap-northeast-2.amazonaws.com/recommender_systems/movielens/datasets"

movie_path = get_file(
    "movies.csv", os.path.join(ROOT_URL, "movies.csv"))
movie_df = pd.read_csv(movie_path)

genre_path = get_file(
    "genres.csv", os.path.join(ROOT_URL, "genres.csv"))
genre_df = pd.read_csv(genre_path)

rating_path = get_file(
    "ratings.csv", os.path.join(ROOT_URL, "ratings.csv"))
rating_df = pd.read_csv(rating_path)


# 메모리 문제를 좀 더 완화하기 위해 8비트로 변경(0~255)
rating_df.rating = (rating_df.rating*2).astype(np.uint8)
rating_df.movie_id = rating_df.movie_id.astype(np.uint32)
rating_df.user_id = rating_df.user_id.astype(np.uint32)

rating_df.drop(columns='rated_at',inplace=True)

### (2) 데이터 샘플링하기

빠른 실습을 위해 영화 중 Top 1000개에 해당하는 영화만 가져와 보도록 하겠습니다.

In [ ]:
top1000_movie_ids = (
#
)

sampling_df = #

# [ 이전 시간에 배운 방식의 한계 ] 
---

현업에서 요구되는 응답 시간은 보통 300ms 이내를 목표로 합니다. 이것보다 늦을 경우, 웹페이지가 느리다고 느끼기 때문에 고객에게 나쁜 서비스 경험을 제공할 수 있습니다. 그래서 추천 시스템을 디자인할 때에는 "얼마나 단시간 내에", "얼마나 많은" 요청을 처리할 수 있는가가 핵심 이슈가 됩니다.



## 1. 연산량이 지나치게 많음

이때까지 디자인한 협업 필터링은 전체 아이템 간의 유사도를 동시에 계산합니다. $O(N^2)$의 관계로 아이템 수가 10배가 늘어나면, 유사도 연산은 100배가 늘어나게 됩니다. 지난 시간까지 다루었던 MovieLens의 경우 아이템이 27278개에 불과했지만, 유통업체 같은 경우 아이템 수가(SKU) 일반적으로 100만개가 훨씬 넘습니다.

100만개만 되더라도 이 경우 기존방식대로 전체 Item Similiarity Matrix을 구하게 되면, 931GB로 일반적인 컴퓨터의 램으로는 계산이 어려운 수준이고 이 규모가 되면 그 때부터 Hadoop과 같은 분산처리에서 다루어야 합니다. 

## 2. 실시간으로 반영이 어려움

그리고 위와 같은 수준의 연산량은 매번 계산하기가 어렵습니다. 그렇기 때문에 보통 주기적으로 하루 단위 혹은 1시간 단위 등 연산을 진행하고, Item Similarity Matrix을 업데이트 하는 방식으로 진행합니다. 뉴스 피드 추천과 같은 컨텐츠 추천에서는 실시간성이 매우 핵심인데, 실시간으로 추천이 갱신되지 못하는 기존의 방식은 추천 시스템으로 적용하기 어렵습니다.

# [ 핵심 개념 ] 
----


## 1. 유사도의 기준 : Jaccard 유사도

Jaccard 유사도는 두 집합 간의 유사도를 계산하는 방식입니다.

$$
sim(A,B) = \frac{A \cap B}{A \cup B}
$$

두 집합이 얼마나 겹쳤냐를 계산하는 유사도로, 이전에 배운 Cosine 유사도나 유클리드 유사도와 달리 Boolean 데이터(구매 유무, 시청 유무, 클릭 유무)에 대한 유사도를 계산할 때 쓰입니다.


reference : [자카드 지수에 대한 Wiki 설명](http://ko.wikipedia.org/wiki/%EC%9E%90%EC%B9%B4%EB%93%9C_%EC%A7%80%EC%88%98)

In [ ]:
movie_A = {1,3,5,6,7} # 1,3,5,6,7번 유저가 본 영화
movie_B = {2,3,4,5} # 2,3,4,5번 유저가 본 영화

#### 자카드 유사도 구하기

In [ ]:
intersection = #
union = #
sim_AB =  #
sim_AB

## 2. 독특한 성질의 Hash, MinHash

MinHash의 특성을 위주로 살펴보도록 하겠습니다. Minhash의 개념에 대한 설명은 아래 Reference를 참조해주시길 바랍니다.

reference : [쉽게 설명한 MinHash  알고리즘](http://blog.haandol.com/2019/05/25/minhash-algorithm-explained.html)

### (1) 집합을 Hash하는 알고리즘 

Minhash는 기본적으로 집합을 Hash하는 알고리즘입니다. Minhash는 집합을 복수개의 Hash Values, 즉 Signature로 표현합니다.

In [ ]:
# Hash하고자 하는 집합
set_A = {"A","B","C","D"}

minhash = #

# 원소 별로 하나씩 minhash에 적용

#
#

위의 과정을 함수로 나타내면 아래와 같습니다.

In [ ]:
def get_hash(target_set, sig_size=128):
    #
    
    return #

MinHash Function은 서로 다른 Hash Function들(위에서는 128개)으로 구성되어 있습니다. MinHash의 알고리즘을 보기 위해 간단한 예제를 만들어보도록 하겠습니다.

In [ ]:
hash_a = 
hash_a

In [ ]:
hash_b = 
hash_b

In [ ]:
hash_c = 
hash_c

MinHash는 각 원소 별로 Signature(여기서는 4개)을 구한 후, 각 Signature 중 가장 작은 값을 저장하는 방식입니다. 가장 작은 값을 저장한다 해서 MinHash라고 불립니다.

In [ ]:
hash_abc = 
hash_abc

In [ ]:
# 각 시그니처 별 최소 값(Min)이 해당 집합 Signature
#

#### 특성 1 : 원소가 중복되면, 동일한 결과 반환한다. 

집합의 특성과 동일하게 이미 Minhash에 포함되었다면 MinHash의 값은 동일하게 나옵니다.

In [ ]:
minhash.update("A".encode('utf8'))

minhash.hashvalues

#### 특성 2: 순서에 영향을 받지 않는다.

그리고 집합과 동일하게, 원소를 update하는 순서가 달라지더라도 동일한 결과를 반환합니다.

In [ ]:
minhash = MinHash()
    
#
minhash.hashvalues

In [ ]:
minhash = MinHash()

#
    
minhash.hashvalues

#### 특성 3 : 집합이 비슷하면, Hash 값도 비슷하게 나온다.

Minhash의 가장 중요한 특성 중 하나로, 집합 간의 IOU 값과 Hash 값의 IOU가 비슷하게 나옵니다.

In [ ]:
set_A = {"A","B","C","D","E","F","G","H"}
set_B = {"D","E","F","G","H","I","J","K"}

#### 두 집합의 IOU 값 구하기 

In [ ]:
intersection = #
union = #
iou = #
iou

#### 두 집합의 MinHash 구하기

In [ ]:
minhash_A = get_hash(set_A)
minhash_B = get_hash(set_B)

In [ ]:
minhash_iou = np.mean(minhash_A == minhash_B)
minhash_iou

MinHash의 signature size가 커질수록 집합 간의 IOU 값과 Hash 값의 IOU가 더 비슷해 집니다.

In [ ]:
sig_size = 256

minhash_A = get_hash(set_A, sig_size)
minhash_B = get_hash(set_B, sig_size)

In [ ]:
np.mean(minhash_A == minhash_B)

## 3. In-Memory DB, Redis

Redis는 대표적인 Key-Value 타입의 In-Memory 데이터베이스입니다. 간단히 설명하면, Python의 Dict와 같이 Key와 Value로 데이터를 저장할 수 있는데, 기존 RDBMS와 달리 RAM에서 데이터를 저장해 훨씬 더 빠르게 입출력을 할 수 있습니다. Redis는 Linux, Mac, Window 모두 현재 지원하고 있으므로, 편하게 로컬 컴퓨터에도 설치가 가능한 DB입니다.

reference : [Redis WIKI 설명](https://en.wikipedia.org/wiki/Redis)

In [ ]:
#

In [ ]:
#

#### 데이터 넣기

In [ ]:
#

#### 데이터 읽기

In [ ]:
#

#### 데이터 수정하기

In [ ]:
#

#### 데이터 제거하기

In [ ]:
#

In [ ]:
#

# \[ MinHash와 In-Memory Cache DB을 활용한 추천 엔진 구현하기 \]
---

## 1. Item 별 Click Stream 구성하기

Click Stream은 이전 시간에 배운 장바구니랑 동일하다고 생각하면 됩니다.

In [ ]:
click_stream = (
    #
)

## 2. MinHash Hashvalues로 바꾸기

각 영화의 MinHash값을 구해보도록 하겠습니다.

In [ ]:
minhash_per_item = #
minhash_per_item

In [ ]:
minhv_df = #
minhv_df.index = #
columns = #
minhv_df.columns = #
minhv_df

이에 대한 각 아이템 간 Item Similarity Matrix를 구하면 아래와 같습니다.

In [ ]:
hash_ious = #
item_sim_df = #
item_sim_df

## 3. Secondary Index로 구성하여 Redis에 저장하기

위와 같이 Item Similarity DataFrame을 세팅하는 경우에는 아이템이 클 경우 불가능해집니다. 대신 Redis에 Secondary Index 방식으로 접근하면 보다 간단하게 할 수 있습니다.

In [ ]:
from redis import Redis

In [ ]:
# redis에 연결
db = Redis('localhost', port=6379)    
#

## 4. 추천 시스템 동작하기

### (1) 아이템 추천하기

In [ ]:
# 해리 포터 : 불의 잔을 본 사람에 대한 추천
movie_df[movie_df.id==40815]

#### 불의 잔 영화에 대한 MinHash Signature 값 가져오기

In [ ]:
target_item = minhv_df.loc[40815]
target_item

#### DB에 Query 전송하기

In [ ]:
%%time

#### 결과 보기

In [ ]:
result = #
result.index = #
result.iloc[:10]

### (2) 정보 갱신하기

#### 예제 상황 
> 유저(1551)번이 해리포터 불의잔(40815)을 보았을 때

In [ ]:
old_minhash = #
old_minhash

In [ ]:
new_minhash = #
new_minhash

MinHash를 갱신하는 것은 추가할 MinHash와 기존 MinHash값을 서로 비교해서 각 Signature의 최소값으로 갱신해주면 됩니다.

In [ ]:
updated_minhash = #
updated_minhash

#### 지워야할 Secondary Index

변경된 Signature만 갱신해주면 됩니다. 아래 값은 이전 minhash 값으로, Secondary Index에서 제거해 주어야 하는 값입니다.

In [ ]:
old_kv = #
old_kv

해당 Secondary Index에는 현재 영화에 대한 movie id가 저장되어 있으므로, 제거해주어야 합니다.

In [ ]:
k = #
v = #

old_index_list = #
print(old_index_list)

위의 리스트에서 영화 불의 잔에 대한 Movie ID인 40815를 제거해 주면 됩니다.

In [ ]:
#
#

#### 추가해야 할 Secondary Index

In [ ]:
new_kv = #
new_kv

새로운 Secondary Index에는 현재 영화에 대한 Movie id를 저장해주어야 합니다. 

In [ ]:
k = #
v = #

new_index_list = #
print(new_index_list)

여기에는 영화 불의 잔에 대한 Movie ID인 40815를 추가해 주면 됩니다.

---

    Copyright(c) 2020 by Public AI. All rights reserved.
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2020/03/30

---